In [1]:
import DRLPDE_main
import time

In [2]:
start = time.time()
DRLPDE_main.maintraining(param='JCPexample6' , use_cuda=True)

end = time.time()
print('Time to train: {:3.1f} seconds'.format(end - start))

Pre-processing: Loading parameters from JCPexample6.py
Initializing the model
Training has begun
No errors in first epoch: Training will continue
step = 50/500, 0.331 s/step, time-to-go:149s
step = 100/500, 0.322 s/step, time-to-go:129s
step = 150/500, 0.319 s/step, time-to-go:112s
step = 200/500, 0.319 s/step, time-to-go:96s
step = 250/500, 0.319 s/step, time-to-go:80s
step = 300/500, 0.319 s/step, time-to-go:64s
step = 350/500, 0.320 s/step, time-to-go:48s
step = 400/500, 0.319 s/step, time-to-go:32s
step = 450/500, 0.318 s/step, time-to-go:16s
step = 500/500, 0.318 s/step, time-to-go: 0s
model saved as savedmodels/JCPexample6.pt
Time to train: 159.2 seconds


In [ ]:
import DRLPDE_postprocessing
DRLPDE_postprocessing.postprocessing(param='JCPexample2',use_model='JCPexample1')

In [7]:
import torch
Test = torch.arange(10, dtype=torch.float)

In [8]:
torch.multinomial(Test, 10)

tensor([6, 5, 7, 3, 4, 9, 8, 1, 2, 0])

In [9]:
Test

tensor([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])

In [13]:
Test2

tensor([ 0.0000,  1.1111,  2.2222,  3.3333,  4.4444,  5.5556,  6.6667,  7.7778,
         8.8889, 10.0000])